# Core

In [ ]:
#| default_exp core
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


This module contains useful utility functions for other modules, as well as the defintion of the library's exceptions class.

### GraphRewriteException

In [ ]:
#| export
class GraphRewriteException(Exception):
    def __init__(self, message="Something went wrong."):
        self.message = message
        super().__init__(self.message)